In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/home/wuxinchao/.conda/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# %pip install openpyxl

In [149]:
# load excel file data
# df = pd.read_excel("~/data/project/pMHC-TCR/seqData/230213_info.xlsx")
file_path = "~/data/project/pMHC-TCR/seqData/230215.xlsx"
df = pd.read_excel(file_path)

In [16]:
df["nSeqHLA"].value_counts()

ATGGCCGTCATGGCGCCCCGAACCCTCCTCCTGCTACTCTCGGGGGCCCTGGCCCTGACCCAGACCTGGGCGGGCTCCCACTCCATGAGGTATTTCTACACCTCCGTGTCCCGGCCCGGCCGCGGGGAGCCCCGCTTCATCGCCGTGGGCTACGTGGACGACACGCAGTTCGTGCGGTTCGACAGCGACGCCGCGAGCCAGAGGATGGAGCCGCGGGCGCCGTGGATAGAGCAGGAGGGGCCGGAGTATTGGGACCAGGAGACACGGAATGTGAAGGCCCAGTCACAGACTGACCGAGTGGACCTGGGGACCCTGCGCGGCTACTACAACCAGAGCGAGGACGGTTCTCACACCATCCAGATAATGTATGGCTGCGACGTGGGGCCGGACGGGCGCTTACTCCGCGGGTACCGGCAGGACGCCTACGACGGCAAGGATTACATCGCCCTGAACGAGGACCTGCGCTCTTGGACCGCGGCGGACATGGCAGCTCAGATCACCAAGCGCAAGTGGGAGGCGGCCCATGCGGCGGAGCAGCAGAGAGCCTACCTGGAGGGCCGGTGCGTGGAGTGGCTCCGCAGATACCTGGAGAACGGGAAGGAGACGCTGCAGCGCACGGACCCCCCCAAGACACATATGACCCACCACCCCATCTCTGACCATGAGGCCACCCTGAGGTGCTGGGCCCTGGGCTTCTACCCTGCGGAGATCACACTGACCTGGCAGCGGGATGGGGAGGACCAGACCCAGGACACGGAGCTCGTGGAGACCAGGCCTGCAGGGGATGGAACCTTCCAGAAGTGGGCGGCTGTGGTGGTGCCTTCTGGAGAGGAGCAGAGATACACCTGCCATGTGCAGCATGAGGGTCTGCCCAAGCCCCTCACCCTGAGATGGGAGCTGTCTTCCCAGCCCACCATCCCCATCGTGGGCATCATTGCTGGCCTGGTTCTCCTTGGAGCTGTGATCACTGGAGCTGTGGTCGCTGCCGTGATGTGGAGGA

In [150]:
# reindex the df by cellname and chain
# df.sort_values(by=['cellname', "chain"])
df = df.set_index(['cellname', "chain"])
# extract the NeoAA, HLA, and aaSeqCDR columns
df = df[["NeoAA", "HLA", "aaSeqCDR1", "aaSeqCDR2", "aaSeqCDR3"]]
# df["aaSeqCDR1"].value_counts()
df["aaSeqCDR"] = df[df.columns[2:]].apply(
    lambda x: '_'.join(x.dropna().astype(str)),
    axis=1
)

In [151]:
idx = pd.IndexSlice

df_a = df.loc[idx[:,"TRA"],]
df_a["AseqCDR"] = df_a["aaSeqCDR"]
df_a.drop(columns=["aaSeqCDR","aaSeqCDR1","aaSeqCDR2","aaSeqCDR3"], inplace=True)
# drop the chain index
df_a.index = df_a.index.droplevel(1)
# print(df_a)
df_b = df.loc[idx[:,"TRB"],]
df_b["BseqCDR"] = df_b["aaSeqCDR"]
df_b.drop(columns=["aaSeqCDR","aaSeqCDR1","aaSeqCDR2","aaSeqCDR3"], inplace=True)
# drop the chain index
df_b.index = df_b.index.droplevel(1)
# print(df_b)

# merge the TRA and TRB dataframes by cellname, HLAs, and NeoAA
df_ab = pd.merge(df_a, df_b, on=["cellname", "HLA", "NeoAA"])
df_ab
# df_ab = pd.merge(df_a, df_b, on="cellname")
# df_ab

/tmp/ipykernel_4163421/3989028287.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a["AseqCDR"] = df_a["aaSeqCDR"]
/tmp/ipykernel_4163421/3989028287.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a.drop(columns=["aaSeqCDR","aaSeqCDR1","aaSeqCDR2","aaSeqCDR3"], inplace=True)
/tmp/ipykernel_4163421/3989028287.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

,NeoAA,HLA,AseqCDR,BseqCDR
cellname,,,,
V350085868_L01_502,VLLSHLSYL,HLA-A*02:01,TSESDYY_QEAYKQQN_CAYRAYMEYGNKLVF,SGHTA_FQGNSA_CASSLGVYEQYF
V350085868_L01_505,VLLSHLSYL,HLA-A*02:01,NIATNDY_GYKTK_CLVGTNSNSGYALNF,LGHNT_FRNRAP_CASGSPDRFEQYF
V350085868_L01_506,VLLSHLSYL,HLA-A*02:01,ATGYPS_ATKADDK_CALTVSYGGSQGNLIF,KGHSH_LQKENI_CASSPFSIGQGLTNNEKLFF
V350085868_L01_507,SLMEQIPHL,HLA-A*02:01,TSGFNG_NVLDGL_CAVKMNTGFQKLVF,DFQATT_SNEGSKA_CSAKLRGSNQPQHF
V350085868_L01_509,VLLSHLSYL,HLA-A*02:01,MNHNS_SASEGT_CASTLRAGWDEQFF,MNHEY_SMNVEV_CASSLSPGGRSEAFF
...,...,...,...,...
V350110758_L02_515,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAAAYYTGANSKLTF,SGHAT_FQNNGV_CASSSPPGVGNEQFF
V350110758_L02_522,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAASGTGTASKLTF,SGHNS_FNNNVP_CASETSNNEQFF
V350110758_L02_523,VVVGAGDVGK,HLA-A*11:01,SSNFYA_MTLNGDE_CAFYGSARQLTF,PRHDT_FYEKMQ_CASSPPTTAANYGYTF


In [152]:
df_ab.to_csv(file_path.replace(".xlsx", ".csv"))

In [85]:
# not used
# (df['aaSeqCDR'].value_counts() > 1).sum()
count = 0
for cell in df.index:
    rep_dict = {}
    if df.loc[(cell), "aaSeqCDR"] in rep_dict:
        count += 1
    else:
        rep_dict[df.loc[(cell), "aaSeqCDR"]] = 1
print(rep_dict)
print(count)

{'SGHTA,FQGNSA,CASSLGVYEQYF': 1}
0


In [47]:
# not used
# sorting data: combine the sequence from the same cell (has the same cellname), 
# add the prefix of the column chain to the columns CDR1, CDR2, CDR3
data = df.groupby("cellname").agg(
    {
        "NeoGeneID": "first",
        "NeoAA" : "first",
        "HLA": "first",
        "chain": lambda x: x.iloc[0] + " " + x.iloc[1],
        "aaSeqCDR1": lambda x: x.iloc[0] + " " + x.iloc[1],
        "aaSeqCDR2": lambda x: x.iloc[0] + " " + x.iloc[1],
        "aaSeqCDR3": lambda x: x.iloc[0] + " " + x.iloc[1],
    }
)
# split each aaSeqCDR1 into 2 columns, one for A chain and one for B chain
data[['aaSeqCDR1_B', 'aaSeqCDR1_A']] = data.aaSeqCDR1.str.split(" ",expand=True,)
data[['aaSeqCDR2_B', 'aaSeqCDR2_A']] = data.aaSeqCDR2.str.split(" ",expand=True,)
data[['aaSeqCDR3_B', 'aaSeqCDR3_A']] = data.aaSeqCDR3.str.split(" ",expand=True,)
data.drop(columns=['aaSeqCDR1', 'aaSeqCDR2', 'aaSeqCDR3'], inplace=True)

In [135]:
# not used
df.groupby(['aaSeqCDR', 'chain', 'cellname']).agg(
    {
        # find the TRA and TRB chains of the same cell has the same aaSeqCDR
        'aaSeqCDR': 'count',
    }
)

In [153]:
df_ab

,NeoAA,HLA,AseqCDR,BseqCDR
cellname,,,,
V350085868_L01_502,VLLSHLSYL,HLA-A*02:01,TSESDYY_QEAYKQQN_CAYRAYMEYGNKLVF,SGHTA_FQGNSA_CASSLGVYEQYF
V350085868_L01_505,VLLSHLSYL,HLA-A*02:01,NIATNDY_GYKTK_CLVGTNSNSGYALNF,LGHNT_FRNRAP_CASGSPDRFEQYF
V350085868_L01_506,VLLSHLSYL,HLA-A*02:01,ATGYPS_ATKADDK_CALTVSYGGSQGNLIF,KGHSH_LQKENI_CASSPFSIGQGLTNNEKLFF
V350085868_L01_507,SLMEQIPHL,HLA-A*02:01,TSGFNG_NVLDGL_CAVKMNTGFQKLVF,DFQATT_SNEGSKA_CSAKLRGSNQPQHF
V350085868_L01_509,VLLSHLSYL,HLA-A*02:01,MNHNS_SASEGT_CASTLRAGWDEQFF,MNHEY_SMNVEV_CASSLSPGGRSEAFF
...,...,...,...,...
V350110758_L02_515,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAAAYYTGANSKLTF,SGHAT_FQNNGV_CASSSPPGVGNEQFF
V350110758_L02_522,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAASGTGTASKLTF,SGHNS_FNNNVP_CASETSNNEQFF
V350110758_L02_523,VVVGAGDVGK,HLA-A*11:01,SSNFYA_MTLNGDE_CAFYGSARQLTF,PRHDT_FYEKMQ_CASSPPTTAANYGYTF


# Load the dataset into torch dataset

In [25]:
# not used
# torch dataset
class TCRDataset(Dataset):
    '''
    Use each two rows of data as a sample (one for alpha chain, one for beta chain), each sample has the common cellname
    Different from the chain, the HLA class is the same for each sample
    The aaSeqCDR1, aaSeqCDR2, aaSeqCDR3 are the CDR1, CDR2, CDR3 of the alpha chain and beta chain, respectively.
    '''
    def __init__(self):
        self.root = "~/data/project/pMHC-TCR/seqData/230213_info.csv"
        

In [154]:
file_path = "~/data/project/pMHC-TCR/seqData/230215.csv"
datasets = pd.read_csv(file_path)
# combine the AseqCDR and BseqCDR into one column with _ as the separator
datasets["seqCDR"] = datasets["AseqCDR"] + "_" + datasets["BseqCDR"]
# drop the AseqCDR and BseqCDR columns
datasets.drop(columns=["AseqCDR", "BseqCDR"], inplace=True)
datasets.index = datasets["cellname"]
datasets.drop(columns=["cellname"], inplace=True)
datasets

,NeoAA,HLA,seqCDR
cellname,,,
V350085868_L01_502,VLLSHLSYL,HLA-A*02:01,TSESDYY_QEAYKQQN_CAYRAYMEYGNKLVF_SGHTA_FQGNSA_...
V350085868_L01_505,VLLSHLSYL,HLA-A*02:01,NIATNDY_GYKTK_CLVGTNSNSGYALNF_LGHNT_FRNRAP_CAS...
V350085868_L01_506,VLLSHLSYL,HLA-A*02:01,ATGYPS_ATKADDK_CALTVSYGGSQGNLIF_KGHSH_LQKENI_C...
V350085868_L01_507,SLMEQIPHL,HLA-A*02:01,TSGFNG_NVLDGL_CAVKMNTGFQKLVF_DFQATT_SNEGSKA_CS...
V350085868_L01_509,VLLSHLSYL,HLA-A*02:01,MNHNS_SASEGT_CASTLRAGWDEQFF_MNHEY_SMNVEV_CASSL...
...,...,...,...
V350110758_L02_515,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAAAYYTGANSKLTF_SGHAT_FQNNGV_CA...
V350110758_L02_522,VVVGAGDVGK,HLA-A*11:01,NSMFDY_ISSIKDK_CAASGTGTASKLTF_SGHNS_FNNNVP_CAS...
V350110758_L02_523,VVVGAGDVGK,HLA-A*11:01,SSNFYA_MTLNGDE_CAFYGSARQLTF_PRHDT_FYEKMQ_CASSP...


In [181]:
# select the first 3-peptide and last 3-peptide of NeoAA sequence
datasets["NeoAA"] = 

SSCMGGMNQR     421
VVGAVGVGK      167
ATAPSLSGK       86
VLLSHLSYL       64
ATATAPSLSGK     42
VVVGAGDVGK      40
VVGAGDVGK       21
TTAPPLSGK       14
VVVGADGVGK      14
SLMEQIPHL        8
LVTDDLLTL        8
SVCAGILSY        1
Name: NeoAA, dtype: int64

In [173]:
X = datasets.drop("HLA", axis=1).values
y = datasets.iloc[:,1].values

In [157]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Test
# use one_hot encoding to HLA subtypes, which are the y label
onehotencoder = OneHotEncoder(handle_unknown="ignore")
X = [["HLA",1], ["HLB",2], ["HLC",1]]
X = onehotencoder.fit_transform(X).toarray()
X

array([[1., 0., 0., 1., 0.],
       [0., 1., 0., 0., 1.],
       [0., 0., 1., 1., 0.]])

In [178]:
onehotEncoder = OneHotEncoder(handle_unknown="ignore")
y = onehotEncoder.fit_transform(y.reshape(-1,1))

In [ ]:
from sklearn.model_selection import train_test_split